# Data Preprocessing for Customer Churn Prediction

This notebook prepares the raw churn dataset for machine learning by:
- Cleaning and standardizing data
- Handling missing values
- Engineering business-driven features
- Encoding and scaling variables
- Saving a model-ready dataset

## 1. Import Required Libraries

Import all necessary Python libraries for data manipulation, preprocessing, and feature engineering.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

import joblib

## 2. Load the Raw Dataset
Load the original, unprocessed dataset to ensure reproducibility and a clean pipeline.

In [2]:
# Load raw dataset
df = pd.read_csv("../data/raw/customer_churn_raw.csv")

# Standardize column names for consistency
df.columns = df.columns.str.strip().str.replace(" ", "_")

# Preview the dataset
df.head()

,PID,CRM_PID_Value_Segment,EffectiveSegment,Billing_ZIP,KA_name,Active_subscribers,Not_Active_subscribers,Suspended_subscribers,Total_SUBs,AvgMobileRevenue,AvgFIXRevenue,TotalRevenue,ARPU,CHURN
0,123759242,Bronze,SOHO,6000.0,VM,2,NaN,NaN,2,40.17,0.0,40.17,NaN,No
1,126145737,Bronze,SOHO,6400.0,VM,3,NaN,NaN,3,40.17,0.0,40.17,13.39,No
2,123506355,Bronze,SOHO,6000.0,DI,2,3.0,NaN,5,40.17,0.0,40.17,20.09,No
3,112595585,Bronze,SOHO,4400.0,MT,1,2.0,NaN,3,40.17,0.0,40.17,40.17,No
4,115097935,Iron,SOHO,4000.0,AD,2,1.0,NaN,3,40.17,0.0,40.17,20.09,No


## 3. Initial Dataset Shape & Sanity Check
- Check dataset dimensions
- Verify column names and data types
- Ensure no unintended changes from the raw file

In [3]:
# Shape of the dataset
df.shape

(8453, 14)

In [4]:
# Dataset information and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8453 entries, 0 to 8452
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PID                     8453 non-null   object 
 1   CRM_PID_Value_Segment   8448 non-null   object 
 2   EffectiveSegment        8453 non-null   object 
 3   Billing_ZIP             8451 non-null   float64
 4   KA_name                 8453 non-null   object 
 5   Active_subscribers      8453 non-null   int64  
 6   Not_Active_subscribers  4304 non-null   float64
 7   Suspended_subscribers   352 non-null    float64
 8   Total_SUBs              8453 non-null   int64  
 9   AvgMobileRevenue        8453 non-null   float64
 10  AvgFIXRevenue           8453 non-null   float64
 11  TotalRevenue            8453 non-null   float64
 12  ARPU                    8452 non-null   float64
 13  CHURN                   8453 non-null   object 
dtypes: float64(7), int64(2), object(5)
memor

## 4. Drop Identifier and Non-Predictive Columns
- Remove unique identifiers such as `PID`
- These columns do not add predictive value and may cause data leakage

In [5]:
df.drop(columns=['PID'], inplace=True, errors='ignore')

## 5. Handle Missing Values

### 5.1 Remove High Missing Value Columns
- Drop `Suspended_subscribers` due to extremely high missing percentage (~96%)
- Such columns add noise and reduce model stability

In [6]:
df.drop(columns=['Suspended_subscribers'], inplace=True)

### 5.2 Impute Logical Zero Values
- Impute missing values in `Not_Active_subscribers` with `0`
- Missing here likely indicates no inactive subscriptions

In [7]:
df['Not_Active_subscribers'].fillna(0, inplace=True)

### 5.3 Drop Rows with Negligible Missing Values
- Remove rows with missing values in:
  - `CRM_PID_Value_Segment`
  - `Billing_ZIP`
  - `ARPU`
- These represent <0.1% of the data and can be safely dropped

In [8]:
df.dropna(subset=[
    'CRM_PID_Value_Segment',
    'Billing_ZIP',
    'ARPU'
], inplace=True)

## 6. Clean & Standardize Categorical Variables

### 6.1 Fix Typographical Errors
- Correct spelling inconsistencies (e.g., `Sliver` → `Silver`)
- Ensures consistent category representation

In [9]:
df['CRM_PID_Value_Segment'] = (
    df['CRM_PID_Value_Segment']
    .replace({'Sliver': 'Silver'})
)

### 6.2 Merge Rare Categories
- Combine rare categories into `OtherSegment`
- Prevents sparse dummy variables during encoding
- Improves model generalization

In [10]:
segment_counts = df['CRM_PID_Value_Segment'].value_counts(normalize=True)
rare_segments = segment_counts[segment_counts < 0.02].index

df['CRM_PID_Value_Segment'] = df['CRM_PID_Value_Segment'].replace(
    rare_segments, 'OtherSegment'
)

### 6.3 Ensure Correct Data Types
- Convert `Billing_ZIP` to categorical (object) type
- ZIP codes represent location, not numeric magnitude

In [11]:
df['Billing_ZIP'] = df['Billing_ZIP'].astype(str)

## 7. Target Variable Encoding
- Convert `CHURN` from categorical (`Yes`/`No`) to numeric (`1`/`0`)
- Required for machine learning models

In [12]:
df['CHURN'] = df['CHURN'].map({'Yes': 1, 'No': 0})

## 8. Feature Engineering
Create new features to capture customer behavior, engagement, and revenue patterns more effectively.

### 8.1 Revenue Intensity per Subscription
- Measures how much revenue each subscription generates

In [13]:
df['Revenue_per_Subscription'] = df['TotalRevenue'] / df['Total_SUBs']

### 8.2 Engagement Score
- Ratio of active subscriptions to total subscriptions
- Captures customer engagement level

In [14]:
df['Engagement_Score'] = (
    df['Active_subscribers'] / df['Total_SUBs']
)

### 8.3 High Inactive Subscriber Flag
- Binary flag indicating presence of inactive subscriptions
- Highlights disengaged customers

In [15]:
df['High_Inactive_Flag'] = np.where(
    df['Not_Active_subscribers'] > 0, 1, 0
)

### 8.4 Mobile Revenue Contribution Ratio
- Measures dependency on mobile services
- Useful since mobile revenue dominates overall revenue

In [16]:
df['Avg_Mobile_to_Total_Revenue_Ratio'] = (
    df['AvgMobileRevenue'] / df['TotalRevenue']
)

### 8.5 FIX Service Usage Indicator
- Binary indicator for FIX (fixed-line) service usage
- FIX users showed higher loyalty in EDA

In [17]:
df['FIX_User_Flag'] = np.where(df['AvgFIXRevenue'] > 0, 1, 0)

### 8.6 Multi-Service Customer Flag
- Identifies customers using multiple services
- Multi-service users tend to churn less

In [18]:
df['Multi_Service_Flag'] = np.where(
    (df['Active_subscribers'] + df['Not_Active_subscribers']) > 1,
    1,
    0
)

### 8.7 High Revenue Customer Flag
- Flags top 10% revenue-generating customers
- Important for business risk analysis

In [19]:
revenue_threshold = df['TotalRevenue'].quantile(0.90)
df['High_Revenue_Flag'] = np.where(
    df['TotalRevenue'] >= revenue_threshold, 1, 0
)

### 8.8 ARPU Category Bucketing
- Bucket ARPU into Low / Medium / High
- Helps linear models capture non-linear churn patterns

In [20]:
df['ARPU_Category'] = pd.qcut(
    df['ARPU'], q=3, labels=['Low', 'Medium', 'High']
)

### 8.9 Customer Revenue Segment
- Bucket customers by total revenue
- Adds interpretable business-level segmentation

In [21]:
df['Customer_Revenue_Segment'] = pd.qcut(
    df['TotalRevenue'],
    q=4,
    labels=['Low', 'Mid', 'High', 'Premium']
)

### 8.10 Customer Value × Engagement Interaction Feature
- Combines revenue intensity and engagement score
- Captures high-value but disengaged customers (high churn risk)

In [22]:
df['Value_Engagement_Index'] = (
    df['Revenue_per_Subscription'] * df['Engagement_Score']
)

### 8.11 Drop Redundant Raw Columns
- Remove raw columns now represented by engineered features
- Reduces multicollinearity

In [23]:
df.drop(columns=[
    'TotalRevenue',
    'AvgMobileRevenue',
    'AvgFIXRevenue'
], inplace=True)

### 8.12 Feature Engineering Summary
- Created revenue, engagement, and service-usage based features
- Converted business metrics into churn-predictive signals
- Removed redundant raw columns to reduce multicollinearity


## 9. Separate Features and Target Variable
- Split dataset into:
  - Feature matrix (X)
  - Target variable (y)

In [24]:
X = df.drop(columns=['CHURN'])
y = df['CHURN']

## 10. Identify Numerical and Categorical Features
- Explicitly list numeric and categorical columns
- Ensures correct preprocessing steps for each type

In [25]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object', 'category']).columns

## 11. Feature Scaling and Encoding

- Numerical features will be standardized using feature scaling, which is essential for distance-based and linear models. 
- Categorical variables will be converted into numerical form using One-Hot Encoding, with the first category dropped to avoid the dummy variable trap.


In [26]:
numeric_transformer = StandardScaler()

categorical_transformer = OneHotEncoder(
    drop='first', handle_unknown='ignore'
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

## 12. Apply and Save Preprocessing Pipeline

- The preprocessing pipeline is applied to the feature set using the defined transformers.
- This step performs numerical feature scaling and categorical feature encoding, producing a model-ready feature matrix.
- The fitted preprocessing pipeline is saved to ensure consistent transformations during model evaluation and future inference.

In [27]:
X_processed = preprocessor.fit_transform(X)

joblib.dump(preprocessor, "../models/preprocessor.pkl")

['../models/preprocessor.pkl']

## 13. Handle Class Imbalance with SMOTE

- Customer churn datasets are typically **class-imbalanced**, which can bias models toward predicting non-churn users  
- **SMOTE (Synthetic Minority Over-sampling Technique)** is applied to balance churn and non-churn classes  
- This enables the model to **learn churn patterns more effectively** and improves **recall for churn customers**  
- The resampled feature matrix is converted back into a **DataFrame with interpretable feature names** for analysis and modeling

In [28]:
# Apply SMOTE to balance classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_processed, y)

# Convert sparse matrix to dense (for analysis & saving)
X_resampled_dense = X_resampled.toarray()

# Restore feature names after preprocessing
feature_names = preprocessor.get_feature_names_out()

# Create final processed DataFrame
processed_df = pd.DataFrame(X_resampled_dense, columns=feature_names)
processed_df['CHURN'] = y_resampled.values

## 14. Final Data Quality & Leakage Check
- Confirm:
  - No missing values remain
  - Target variable is not leaked into features
  - Dataset shape is consistent

In [29]:
X_resampled.shape, y_resampled.value_counts(normalize=True)

((15792, 490),
 CHURN
 0    0.5
 1    0.5
 Name: proportion, dtype: float64)

## 15. Save the Engineered Dataset
- Save the fully processed dataset for modeling
- Ensures reproducibility and clean separation of stages

In [30]:
# Converting sparse matrix → dense array BEFORE DataFrame
X_dense = X_resampled.toarray()

processed_df = pd.DataFrame(X_dense)
processed_df['CHURN'] = y_resampled.values

# Saving
processed_df.to_csv(
    "processed_churn_data.csv",
    index=False
)

## Final Output
- Dataset is fully cleaned, engineered, balanced, and model-ready
- Ready for training